In [12]:
from datetime import datetime

import os
import csv
import random

In [13]:

# Loading csv files

def load_expenses():
    expenses = []
    Ex_file_path= 'data/expenses.csv'

    try:
        with open(Ex_file_path, 'r') as file:
            csv_reader = csv.DictReader(file)

            for row in csv_reader:
                expenses.append(row)

    except FileNotFoundError:
         print(f'File not found')

    except Exception as e:
         print(e)   

    return expenses 


def load_categories():
    categories = []
    Cat_file_path = 'data/categories.csv'
    
    try:
        with open(Cat_file_path,'r' ) as file:
            csv_reader = csv.DictReader(file)

        for row in csv_reader:
            categories.append(row)

    except FileNotFoundError:
         print(f'File not found')

    except Exception as e:
         print(e)  

    return categories


def load_budgets():
    budgets = []
    Budg_file_path = 'data/budgets.csv'
    
    try:
        with open(Budg_file_path, 'r') as file:
            csv_reader = csv.DictReader(file)

        for row in csv_reader:
            budgets.append(row)

    except FileNotFoundError:
         print(f'File not found')

    except Exception as e:
         print(e)           

    return budgets
                     

In [14]:
def save_expenses(expense_list, overwrite= False):
    csv_header = ['transaction_id','date','category','description','amount','payment_method']
    Save_Ex_filePath = 'data/expenses.csv'

    existing_Ex = load_expenses()

    with open(Save_Ex_filePath, 'w') as file:
        csv_writer = csv.DictWriter(file, fieldnames=csv_header)

        csv_writer.writeheader()

        all_expenses = existing_Ex + expense_list

        if overwrite == True:
            all_expenses = expense_list

        csv_writer.writerows(all_expenses)

        print(all_expenses,len(all_expenses))

    return True


def save_budgets(budget_list):
    csv_header = ['category','monthly_limit']
    Save_Budg_filepath = 'data/budgets.csv'
    
    existing_budg = load_budgets()

    with open(Save_Budg_filepath, 'w') as file:
        csv_writer = csv.DictWriter(file, fieldnames=csv_header)

        csv_writer.writeheader()

        all_budgets = existing_budg + budget_list

        csv_writer.writerows(all_budgets)

    return True 

def save_categories(category_list):
    csv_header = ['category_name','description']
    Save_Cat_filepath = 'data/categories.csv'

    existing_Cat = load_categories()

    with open(Save_Cat_filepath, 'w') as file:
        csv_writer = csv.DictWriter(file, fieldnames=csv_header)

        csv_writer.writeheader()

        all_categories = existing_Cat + category_list

        csv_writer.writerows(all_categories)

    return True    
    

In [15]:
class Expense:
    
    def __init__(self, transaction_id, date, category, description, amount, payment_method):
        self.transaction_id = transaction_id
        self.date = date
        self.category = category
        self.description = description
        self.amount = amount
        self.payment_method = payment_method


    def to_dict(self):
        expense_dict = {'transaction_id':self.transaction_id,
                        'date':self.date,
                        'category':self.category,
                        'description':self.description,
                        'amount':self.amount,
                        'payment_method':self.payment_method}
        
        return expense_dict
        
        
    def __str__(self):
        return f'''Transaction ID: {self.transaction_id}
                Date: {self.date} 
                Category: {self.category} 
                Description: {self.description} 
                amount: {self.amount}',
                Payment Method:{self.payment_method}'''
    
    def validate(self):
        valid_payment_method = ['card', 'transfer', 'cash']
        valid_categories = ['Entertainment', 'Transport', 'Healthcare', 'Food',
                            'Education','Other', 'Utilities', 'Others', 'Shopping', 'Pets' ]
        
        is_payment_method_valid = False
        is_category_valid = False
        is_amount_valid =False

        #validate payment method
        if self.payment_method in valid_payment_method:
            is_payment_method_valid = True

        if self.category in valid_categories:
            is_category_valid = True    

        #validate amount    
        if self.amount > 0:
            is_amount_valid = True   
   

        return is_payment_method_valid and is_category_valid and is_amount_valid    


In [26]:
def add_expense(expenses_list, categories_list):
    date_input = input('Enter date in YYYY-MM-DD format:')
    date= None
    amount = 0
    
    #get and validate date
    try:
        valid_date = datetime.strptime(date_input, '%Y-%m-%d')
        date = valid_date 
    except ValueError as e:
        print('Not a valid date:', e)

    #display and select category
    for index, category in enumerate(categories_list):
        print(f'index: {index}, Value:{category}') 

    cat_input = input('Enter category: ') 

    #get description
    description = input('Enter description: ')

    #get and validate amount
    try:
        amount_input = float(input('Enter amount: '))
        if amount_input >= 0:
            amount = amount_input
            
        else: 
            raise Exception('Invalid Input')
    except ValueError as e:
        print('Invalid amount:', e)

    #get payment method
    payment_method = input('Enter payment method:')

    valid_PM = ['card', 'cash', 'transfer']

    if payment_method not in valid_PM:

        raise Exception('Payment method not valid')
    
    #Expense Object
    transaction_id = random.randint(100000, 999999)
    expense = Expense(transaction_id, date, cat_input,
                     description,amount, payment_method)    
    
    #Add to list and save
    expense_dict = expense.to_dict()
    new_ex_list = expenses_list.append(expense_dict)

    save_expenses([expense_dict])

    print(f'Expense added Successfully: {transaction_id} ')


#add_expense([],['Shopping','Food', 'Healthcare'])


In [17]:
def display_expenses(expenses_list, title="All Expenses"):
    #Display expenses in formatted table
    print(f'=== {title} ===')
    headers =['transaction_id','date','category',
              'description','amount','payment_method']
    
    print(" | ".join(headers))
    print("-" * (len(" | ".join(headers))))  
    
    total_ex_amount = 0
    for expense in expenses_list:
            total_ex_amount += float(expense.get('amount'))
            print(" | ".join(expense.values())) 

    print(f' Total Expense Amount: {total_ex_amount}' )            

def filter_by_date_range(expenses_list, start_date, end_date):
    #Return expenses between two dates
    filtered_expense = []

    for expense in expenses_list:
        expense_date = expense.get('date')

        exp_date_obj =datetime.strptime(expense_date, "%Y-%m-%d").date()
        start_dt_obj = datetime.strptime(start_date, "%Y-%m-%d").date() 
        end_dt_obj = datetime.strptime(end_date, "%Y-%m-%d").date() 

        if start_dt_obj <= exp_date_obj <= end_dt_obj:
             filtered_expense.append(expense)
    
    display_expenses(filtered_expense)


def filter_by_category(expenses_list, category):
    #Return expenses for specific category
    filtered_expense = []
    
    for expense in expenses_list:
        expense_category = expense.get('category')

        if expense_category == category:
             filtered_expense.append(expense)

    display_expenses(filtered_expense)
         
    

def search_expenses(expenses_list, keyword):
    #Search expenses by description
    filtered_expense = []

    for expense in expenses_list:
        expense_description = expense.get('description')

        if keyword in expense_description:
             filtered_expense.append(expense) 

    display_expenses(filtered_expense)         
         

#expenses = load_expenses()
#search_expenses(expenses,'dinner' )

In [18]:
def edit_expense(expenses_list, transaction_id):
    #Edit an existing expense
    #Find expense by ID
    item_to_edit = None
    edit_index = None

    for index, item in enumerate(expenses_list):
        current_item_T_ID = item.get('transaction_id')
        if current_item_T_ID == transaction_id:
            item_to_edit = item
            edit_index = index

    if item_to_edit == None:
        raise Exception(f'item not found {transaction_id}')

    #Display current values
    display_expenses([item_to_edit])

    #Prompt for new values (allow skip)
    input_description= input('Enter description: ')
    input_amount = float(input('Enter amount: '))

    item_to_edit.update({'description': input_description, 'amount': input_amount})

    #Update expense
    expenses_list[edit_index] = item_to_edit

    #Save changes
    save_expenses(expenses_list, True)

def delete_expense(expenses_list, transaction_id):
    """Delete an expense after confirmation"""
    #Find expense
    item_to_del = None
    del_index = None

    for index, item in enumerate(expenses_list):
        current_item_T_ID = item.get('transaction_id')
        if current_item_T_ID == transaction_id:
            item_to_del = item
            del_index = index

    if item_to_del == None:
        raise Exception(f'item not found {transaction_id}')
    #Display details
    display_expenses([item_to_edit])

    #Confirm deletion
    input_del = input('Type Delete to confirm delete' )
    if input_del == 'delete':
        del expenses_list[del_index] 
    else:
        raise Exception(f'Invalid Input: {input_del}')

    #Save changes
    save_expenses(expenses_list, True)
   

In [35]:
def display_main_menu():
    #Display main menu and get user choice
    print("\n" + "="*50)
    print("     PERSONAL EXPENSE TRACKER")
    print("="*50)
    print("\n[1] Add New Expense")
    print("[2] View Expenses")
    print("[3] Edit Expense")
    print("[4] Delete Expense")
    print("[0] Exit")
    print("\n" + "-"*50)
    
    choice = input("Enter your choice: ")
    return choice

def view_expenses_menu(expenses):
    print('[1] Display Expense')
    print('[2] Filter By Date Range')
    print('[3] Filter By Category')
    print('[4] Search Expense')
    print('[0] Exit')

    while True:
        choice = input("Enter display menu choice: ")

        if choice == '1':
            display_expenses(expenses)
        elif choice == '2':
            start_date = input('Enter start date: ')
            end_date = input('Enter end date: ')
            filter_by_date_range(expenses, start_date, end_date)
        elif choice =='3':
            cate_input = input('Enter a category: ') 
            filter_by_category(expenses, cate_input)
        elif choice == '4':
            search_input = input('Enter Keyword: ')
            search_expenses(expenses, search_input)  
        else:
            break  

             
    
def main():
    """Main program loop"""
    # Initialize data
    expenses = load_expenses()
    budgets = load_budgets()
    categories = load_categories()
    
    while True:
        choice = display_main_menu()
        
        if choice == '1':
            add_expense(expenses, categories)
        elif choice == '2':
            view_expenses_menu(expenses)
        elif choice == '3':
            T_ID_input = input('Enter transaction ID: ')
            edit_expense(expenses, T_ID_input)
        elif choice == '4':
            del_input = input('Enter transaction ID: ')
            delete_expense(expenses, del_input)
        elif choice == '0':
            print("\nThank you for using Expense Tracker!")
            break
        else:
            print("\n❌ Invalid choice. Please try again.")

In [ ]:
main()

I/O operation on closed file.
I/O operation on closed file.

     PERSONAL EXPENSE TRACKER

[1] Add New Expense
[2] View Expenses
[3] Edit Expense
[4] Delete Expense
[0] Exit

--------------------------------------------------
=== All Expenses ===
transaction_id | date | category | description | amount | payment_method
------------------------------------------------------------------------
451813 | 2008-08-09 | Healthcare |  | 0.0 | cash
 Total Expense Amount: 0.0
